In [1]:
import dask.dataframe as dd
import pandas as pd
!pip install fastparquet
from fastparquet import ParquetFile
pd.set_option('display.max_columns', None)

In [2]:
!pip install pytorch-pretrained-bert

In [0]:
import re
from scipy.spatial.distance import cosine
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

import logging

import matplotlib.pyplot as plt
% matplotlib inline

# Load the dataset


In [5]:
#Loading reviews data
reviews = dd.read_parquet('https://tally-ai-dspt3.s3.amazonaws.com/yelp-restaurants/reviews.parquet.gzip')
reviews.head()

,review_id,business_id,reviews_stars,date,text
index,,,,,
0,ipufYjKx5saLVuDR6f6o0w,S9RoY_Smsh0a2JPo90bkdg,5,2017-08-21 02:03:08,Was there with a girlfriend at lunch time toda...
1,VNOhF-xUYguSjA01yx5nwA,S9RoY_Smsh0a2JPo90bkdg,2,2017-08-20 17:46:40,I thought my meal was OK. I ordered an omelett...
2,PMXYXphbbkx2TM80AtIpQQ,S9RoY_Smsh0a2JPo90bkdg,2,2015-12-10 17:53:39,It's just a breakfast not too fancy. Weak coff...
3,7hxilsquX34WXHYc80qLkw,S9RoY_Smsh0a2JPo90bkdg,1,2017-09-02 18:54:37,"Shoule be zero stars! So, we have been here a ..."
4,Xnf39DsDWqiv1wulL2NK9g,S9RoY_Smsh0a2JPo90bkdg,5,2017-04-30 05:33:08,I LOVE this place!!! I'm so happy they opened ...


In [6]:
#Loading business data
business = dd.read_csv('https://tally-ai-dspt3.s3.amazonaws.com/yelp-restaurants/business.csv', blocksize=None)
business.head()

,Unnamed: 0,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,state,business_stars
0,8,404 E Green St,"{'RestaurantsAttire': ""u'casual'"", 'Restaurant...",pQeaRpvuhoEqudo3uymHIQ,"Ethnic Food, Food Trucks, Specialty Food, Impo...",Champaign,"{'Monday': '11:30-14:30', 'Tuesday': '11:30-14...",1,40.110446,-88.233073,The Empanadas House,61820,5,IL,4.5
1,20,4508 E Independence Blvd,"{'RestaurantsGoodForGroups': 'True', 'OutdoorS...",CsLQLiRoafpJPJSkNX2h5Q,"Food, Restaurants, Grocery, Middle Eastern",Charlotte,NaN,0,35.194894,-80.767442,Middle East Deli,28205,5,NC,3.0
2,24,"15480 Bayview Avenue, unit D0110","{'RestaurantsTableService': 'False', 'Restaura...",eBEfgOPG7pvFhb2wcG9I7w,"Restaurants, Cheesesteaks, Poutineries",Aurora,"{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",1,44.010962,-79.448677,Philthy Phillys,L4G 7J1,4,ON,4.5
3,25,300 John Street,"{'GoodForKids': 'True', 'RestaurantsTakeOut': ...",lu7vtrp_bE9PnxWfA8g4Pg,"Japanese, Fast Food, Food Court, Restaurants",Thornhill,NaN,1,43.820492,-79.398466,Banzai Sushi,L3T 5W4,7,ON,4.5
4,30,13071 Yonge Street,"{'Ambience': ""{'touristy': False, 'hipster': F...",9sRGfSVEfLhN_km60YruTA,"Persian/Iranian, Turkish, Middle Eastern, Rest...",Richmond Hill,"{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21...",1,43.947011,-79.454862,Apadana Restaurant,L4E 1A5,3,ON,3.0


In [0]:
#Conver dd to pd dataframe
reviews = reviews.compute()

In [10]:
# Load pre-trained model tokenizer(vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

100%|██████████| 231508/231508 [00:00<00:00, 886977.37B/s]


In [15]:
# locate the records containing "awesome", "strong" and "great"
reviews_tmp = reviews[reviews['text'].str.contains("^(?=.*awesome)(?=.*strong)(?=.*great).+",
                                   flags=re.IGNORECASE, regex=True)]
print(reviews_tmp.shape)
reviews_tmp

(711, 5)


,review_id,business_id,reviews_stars,date,text
index,,,,,
1210,WpRodqG0U2RsZOuLu0vOSQ,7sb2FYLS2sejZKxRYF9mtg,5,2014-09-25 04:04:31,I literally have no complaints. It was a Tuesd...
1551,0rCuo-QSDvSAEfBYwuDswg,7sb2FYLS2sejZKxRYF9mtg,5,2018-03-23 06:34:08,I read the reviews and knew I wanted to try th...
2815,1asOQC6tws-CwA6OGcmgeg,7sb2FYLS2sejZKxRYF9mtg,5,2013-08-14 22:10:28,If you are looking for fresh sushi restaurant ...
4886,nUp7QbvBBsWINKKItVrM2A,0PMKMyfX1j7Nc4esk1RaMg,4,2009-01-13 15:15:57,We tried Humble Pie for takeout. The pizza wa...
4952,RzQdHQh3Ra2SFVbJa9d0ww,0PMKMyfX1j7Nc4esk1RaMg,5,2011-07-10 12:02:02,"Truth be told, great happy hour. Very strong d..."
...,...,...,...,...,...
11333,mLIPwI5Sd8kUdh3JN3GfVA,UAGZsl9ul16iDHmFNDLj4A,5,2016-12-14 16:16:31,For a little dive bar this place is awesome! I...
1072,7ELDahnkBUrkrjzunLUkVw,BYtGiZr3dbxa2qAhg_3I4w,5,2014-05-13 02:28:40,"I'll keep it short and sweet... great service,..."
1579,KzWdoutmdg3CHCttdAIe6g,DYuOxkW4DtlJsTHdxdXSlg,3,2017-12-08 08:34:37,I haven't been here in about 10 years decided ...


In [17]:
text = reviews.loc[23828, 'text']
marked_text = "[CLS] " + text + " [SEP]"

# Tokenizer our sentence with the Bert tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print(tokenized_text)

TypeError: ignored